## Scikit-Learn Hierarchical Clustering
### Using MALL_CUSTOMERS_VIEW from SAP Datasphere. This view has 200 records

## Install fedml_azure package

In [ ]:
pip install fedml_azure --force-reinstall

## Import the libraries needed in this notebook

In [ ]:
from fedml_azure import DwcAzureTrain

## Set up
### Creating a Training object and setting the workspace, compute target, and environment.

Before running the below cell, ensure that you have a workspace and replace the subscription_id, 
resource_group, and workspace_name with your information.

The whl file for the fedml_azure library must be passed to the pip_wheel_files key in the environment_args and 
to use scikit-learn, you must pass the name to conda_packages as well.

In [ ]:
#creation of training object and creating workspace in constructor.

training = DwcAzureTrain(
                       workspace_args={"subscription_id": '<subscription_id>',
                                        "resource_group": '<resource_group>',
                                        "workspace_name": '<workspace_name>'
                                        },
                        experiment_args={'name':'test-2'},
                        environment_type='CondaPackageEnvironment',
                        environment_args={'name':'test-env-hc', 'conda_packages':['scikit-learn'],'pip_packages':['fedml_azure']},
                        compute_type='AmlComputeCluster',
                        compute_args={'vm_size':'Standard_D12_v2',
                                'vm_priority':'lowpriority',
                                'compute_name':'cpu-clu-hc',
                                'min_nodes':0,
                                'max_nodes':1,
                                'idle_seconds_before_scaledown':1700
                                })


### Then, we need to generate the run config. This is needed to package the configuration specified so we can submit a job for training. 

Before running the following cell, you should have a config.json file with the specified values to allow you to access to SAP Datasphere. Provide this file path to config_file_path in the below cell.

You should also have the follow view MALL_CUSTOMERS_VIEW created in your SAP Datasphere. To gather this data, please refer to https://www.kaggle.com/roshansharma/mall-customers-clustering-analysis/data

https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py

In [ ]:
#generating the run config
src=training.generate_run_config(config_file_path='dwc_configs/config.json',
                          config_args={
                                          'source_directory':'Scikit-Learn-Hierarchical-Clustering',
                                          'script':'train_script.py',
                                          'arguments':['--model_file_name','regression.pkl', '--table_name', 'MALL_CUSTOMERS_VIEW', '--table_size', '1']
                                          }
                            )

### Submitting the job for training

In [ ]:
#submitting the training run
run=training.submit_run(src)

## Register the model for deployment

In [ ]:
model = training.register_model(run=run,
                           model_args={'model_name':'sklearn_hcluster_model', 
                           'model_path':'outputs/regression.pkl'},
                           resource_config_args={"cpu":1, "memory_in_gb":0.5},
                           is_sklearn_model=True)

print('Name:', model.name)
print('Version:', model.version)